# Experiment 05-07

In [ ]:
import sys
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information
import logging
from pqdm.threads import pqdm
from tqdm.notebook import tqdm

## Configure Logging

In [ ]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("ex05_07.log"),
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2022-07-13 15:46:14 INFO     Started


## Show the available Platforms

In [ ]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: AMD Accelerated Parallel Processing
Platform 0 - Vendor: Advanced Micro Devices, Inc.
Platform 0 - Version: OpenCL 2.1 AMD-APP (3354.13)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: gfx1010:xnack-
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1675 Mhz
 Device - Compute Units: 18
 Device - Local Memory: 64 KB
 Device - Constant Memory: 6893568 KB
 Device - Global Memory: 8 GB
 Device - Max Buffer/Image Size: 6732 MB
 Device - Max Work Group Size: 256


 --------------------------------------------------------
 Device - Name: gfx1010:xnack-
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1815 Mhz
 Device - Compute Units: 20
 Device - Local Memory: 64 KB
 Device - Constant Memory: 6893568 KB
 Device - Global Memory: 8 GB
 Device - Max Buffer/Image Size: 6732 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [ ]:
platform_number = 0
device_number = 1

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

In [ ]:
cl_device = cl_devices[device_number]

cl_threads = cl_device.max_compute_units * cl_device.max_work_group_size
if cl_device.type & 4 == 0:
    cl_threads = cl_devices.max_work_group_size

print('Max Compute Units:', cl_device.max_compute_units)
print('Max Compute Units:', cl_device.max_work_group_size)
print('Threads:', cl_threads)

Max Compute Units: 20
Max Compute Units: 256
Threads: 5120


## Compile the Program

In [ ]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [ ]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

c:\Users\johnny\env\dsm070\lib\site-packages\pyopencl\__init__.py:274: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Kernel Names: get_random_numbers;mine_eight;get_random_string;get_single_hash_nonce;get_single_hash;hash_main;mine_eight_sequential;mine_nonce


## Mine Nonce

In [ ]:
max_nonce = 16

In [ ]:
def display_output(zeros_found : dict, nonce : np.ndarray,
                   nonce_len : np.ndarray, plaintext : str, prefix : str) -> None:
    """
    Display the output generated by the OpenCL Kernel.
    """
    for i in range(0, 64):
        if nonce_len[i] > 0:
            nonce_str = prefix + nonce[i * max_nonce:i * max_nonce + nonce_len[i]].tobytes().decode('UTF-8')
            zeros_found[i] = nonce_str
            hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))

            logging.info("%4d: [%2d] %32s %64s" % (i, len(nonce_str), nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))

In [10]:
postfix = "FOULDS-"
base_text = 'this is a description of the latest block'
plaintext = base_text + postfix
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)

# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
window_size = np.uint32(3000000)
nonce = np.zeros(shape=max_nonce * 64, dtype=np.uint8)
nonce_len = np.zeros(shape=64, dtype=np.uint8)
#hash_count = np.zeros(shape=1, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)
#cl_hash_count = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, hash_count.nbytes)

zeros_found = {}

keep_running = True
while (keep_running):
    seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
    cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)

    # execute the program
    cl_program.mine_eight(
        cl_queue, (cl_threads,), None,
        cl_seed,
        cl_window_size,
        cl_plaintext_bytes,
        cl_plaintext_length,
        cl_nonce,
        cl_nonce_len,
    #    cl_hash_count
    )

    # get the results
    cl.enqueue_copy(cl_queue, nonce, cl_nonce)
    cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)
    #cl.enqueue_copy(cl_queue, hash_count, cl_hash_count)

    # display the results
    #print('Hash Count:', hash_count[0])
    display_output(zeros_found, nonce, nonce_len, base_text, postfix)

    keep_running = True
    logging.info('Iteration complete.')

2022-07-13 15:46:16 INFO     build program: kernel 'mine_eight' was part of a lengthy cache retrieval (0.48 s)
2022-07-13 15:47:03 INFO     Iteration complete.
2022-07-13 15:47:50 INFO        0: [22]           FOULDS-9`u4>"O3q#KIffl 000000008603c923b421b5ac2caffd06fc0bd076b605b92a01adc36564d8d610
2022-07-13 15:47:50 INFO     Iteration complete.
2022-07-13 15:48:37 INFO     Iteration complete.
2022-07-13 15:49:24 INFO        0: [12]                     FOULDS-zKH&U 000000002d1b1f66411ba7872c7b5171220129253a670abdda228317b98be26e
2022-07-13 15:49:24 INFO     Iteration complete.
2022-07-13 15:50:11 INFO        0: [15]                  FOULDS-@d*]:E7j 00000000200f836f68161694fd45b7640f5750af7e192f9e061fd56a1f8dca60
2022-07-13 15:50:11 INFO     Iteration complete.
2022-07-13 15:50:58 INFO        0: [12]                     FOULDS-cHzyy 00000000d6920e44c98c25c40132444e2098023904bf20fc9230e8e882462325
2022-07-13 15:50:58 INFO     Iteration complete.
2022-07-13 15:51:46 INFO        0: [17]    

In [ ]:
# def find_zeros(i : int) -> list:
#     """
#     Find the zeros in the zeros_found dictionary.
#     """
#     output = []

#     iterations = 1000000
#     min_len = 6
#     nonce = i * iterations

#     for k in range(1, iterations):
#         hash = hashlib.sha256((plaintext+str(nonce)).encode('utf-8')).hexdigest()
#         zeros = len(hash) - len(hash.lstrip('0'))
#         if zeros > min_len:
#             output.append('%16d %s\n' % (nonce, hash))

#         nonce += 1

#     return output   

# plaintext = 'this is a description of the latest block'
# n_jobs=256
# results = pqdm(range(n_jobs), function=find_zeros, n_jobs=n_jobs, argument_type='iterator')
# for result in results:
#     if len(result) > 0:
#         print(result)
